In [ ]:
import pandas as pd

df = pd.read_csv('find_youtube_urls.csv')

df

,Subject,Song Title,Artists,YouTube URL,Video Type,Error
0,"Die With A Smile - Lady Gaga, Bruno Mars",Die With A Smile,"Lady Gaga, Bruno Mars",https://www.youtube.com/watch?v=kPa7bsKwL-c,Official Music Video,NaN
1,BIRDS OF A FEATHER - Billie Eilish,BIRDS OF A FEATHER,Billie Eilish,https://www.youtube.com/watch?v=V9PVRfjEBTI,Official Music Video,NaN
2,"APT. - ROSÉ, Bruno Mars","APT. - ROSÉ, Bruno Mars","ROSÉ, Bruno Mars",https://www.youtube.com/watch?v=ekr2nIex040,Official Music Video,NaN
3,Ordinary - Alex Warren,Ordinary,Alex Warren,https://www.youtube.com/watch?v=u2ah9tWTkmk,Official Music Video,NaN
4,DtMF - Bad Bunny,DtMF,Bad Bunny,https://www.youtube.com/watch?v=nu_ift92Ipc,Topic Audio,NaN
5,back to friends - sombr,back to friends,sombr,https://www.youtube.com/watch?v=c8zq4kAn_O0,Official Music Video,NaN
6,"Golden - HUNTR/X, EJAE, AUDREY NUNA, REI AMI, ...",Golden,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo...",https://www.youtube.com/watch?v=yebNIHKAC4A,Official Lyric Video,NaN
7,"luther (with sza) - Kendrick Lamar, SZA",luther (with sza),"Kendrick Lamar, SZA",https://www.youtube.com/watch?v=sNY_2TEmzho,Official Music Video,NaN
8,That's So True - Gracie Abrams,That's So True,Gracie Abrams,https://www.youtube.com/watch?v=W_YOJWZIjxo,Topic Audio,NaN
9,WILDFLOWER - Billie Eilish,WILDFLOWER,Billie Eilish,https://www.youtube.com/watch?v=O1PkZaFy61Y,Topic Audio,NaN


In [ ]:
import pandas as pd
import json
import time
from itertools import islice
from youtube_comment_downloader import YoutubeCommentDownloader, SORT_BY_POPULAR

# Load the CSV
df = pd.read_csv('find_youtube_urls.csv')

# Config
MAX_COMMENTS = 310000
OUTPUT_FILE = 'youtube_comments_2.json'

downloader = YoutubeCommentDownloader()
all_comments = []

# Calculate how many comments per video (distribute evenly)
urls = df['YouTube URL'].dropna().tolist()
num_videos = len(urls)
comments_per_video = MAX_COMMENTS // num_videos
print(f"Scraping up to {comments_per_video} comments per video across {num_videos} videos...")
print(f"Target total: {MAX_COMMENTS} comments\n")

for idx, row in df.iterrows():
    url = row.get('YouTube URL')
    title = row.get('Song Title', 'Unknown')
    artists = row.get('Artists', 'Unknown')

    if pd.isna(url) or not url.strip():
        print(f"  Skipping row {idx} — no URL")
        continue

    print(f"[{idx + 1}/{len(df)}] {title} — {artists}")
    print(f"  URL: {url}")

    try:
        comments = downloader.get_comments_from_url(url, sort_by=SORT_BY_POPULAR)
        count = 0
        for comment in islice(comments, comments_per_video):
            comment['song_title'] = title
            comment['artists'] = artists
            comment['youtube_url'] = url
            all_comments.append(comment)
            count += 1

        print(f"  ✓ Collected {count} comments")
    except Exception as e:
        print(f"  ✗ Error: {e}")

    # Brief pause to avoid rate limiting
    time.sleep(1)

print(f"\nTotal comments collected: {len(all_comments)}")

# Save to JSON
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(all_comments, f, ensure_ascii=False, indent=2)

print(f"Saved to {OUTPUT_FILE}")


Scraping up to 10000 comments per video across 31 videos...
Target total: 310000 comments

[1/31] Die With A Smile — Lady Gaga, Bruno Mars
  URL: https://www.youtube.com/watch?v=kPa7bsKwL-c
